In [1]:
import yaml, logging, sys, os
from pyvis.network import Network
from IPython.display import display
from llama_index.llms import AzureOpenAI
from llama_index.llm_predictor import LLMPredictor
from llama_index import set_global_service_context
from llama_index.vector_stores import Neo4jVectorStore
from llama_index.text_splitter import TokenTextSplitter
from llama_index.embeddings import HuggingFaceEmbedding
from llama_index import (
                        StorageContext,
                        VectorStoreIndex,
                        SimpleDirectoryReader, 
                        load_graph_from_storage,
                        load_index_from_storage,
                        ServiceContext,
                        PromptHelper
                        )

logging.basicConfig(
                    stream=sys.stdout, 
                    level=logging.INFO
                    )
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))


In [2]:
with open('cadentials.yaml') as f:
    credentials = yaml.load(f, Loader=yaml.FullLoader)

# Configuring LLMs

In [3]:
embedding_llm = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")
llm=AzureOpenAI(
                model=credentials['AZURE_ENGINE'],
                api_key=credentials['AZURE_OPENAI_API_KEY'],
                deployment_name=credentials['AZURE_DEPLOYMENT_ID'],
                api_version=credentials['AZURE_OPENAI_API_VERSION'],
                azure_endpoint=credentials['AZURE_OPENAI_API_BASE']
                )
chat_llm = LLMPredictor(llm)

prompt_helper = PromptHelper(
                            num_output=256,
                            context_window=4096,
                            chunk_overlap_ratio=0.1,
                            chunk_size_limit=None
                            )

text_splitter = TokenTextSplitter(
                                separator=" ",
                                chunk_size=1024,
                                chunk_overlap=20,
                                backup_separators=["\n"]
                                )

service_context = ServiceContext.from_defaults(
                                                text_splitter=text_splitter,
                                                prompt_helper=prompt_helper,
                                                embed_model=embedding_llm,
                                                llm_predictor=chat_llm
                                                )

set_global_service_context(service_context)

/Users/1zuu/miniforge3/envs/llamaindex/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


LLMPredictor is deprecated, please use LLM instead.


# Load and Store Data

In [4]:
# Load Data
documents = SimpleDirectoryReader("./data/langchain_blog_posts").load_data()
print(len(documents))

52


In [5]:
if not os.path.exists('./db/04/vector/'):
    vec_index = VectorStoreIndex.from_documents(
                                                documents,
                                                service_context = service_context
                                                )
    vec_index.storage_context.persist(persist_dir='./db/04/vector/')
else:
    storage_context_vector = StorageContext.from_defaults(persist_dir='./db/04/vector/')
    vec_index = load_index_from_storage(storage_context=storage_context_vector)

INFO:llama_index.indices.loading:Loading all indices.
Loading all indices.


In [6]:
neo4j_vector = Neo4jVectorStore(
                                credentials['NEO4J_USERNAME'], 
                                credentials['NEO4J_PASSWORD'], 
                                credentials['NEO4J_URI'], 
                                384
                                )

if not os.path.exists('./db/04/graph/'):
    storage_context_graph = StorageContext.from_defaults( graph_store=neo4j_vector)
    
    graph_index = VectorStoreIndex.from_documents(
                                                documents, 
                                                storage_context=storage_context_graph
                                                )
    
    graph_index.storage_context.persist(persist_dir='./db/04/graph/')
    
else:
    storage_context_graph = StorageContext.from_defaults(
                                                        graph_store=neo4j_vector,
                                                        persist_dir='./db/04/graph/'
                                                        )
    graph_index = load_index_from_storage(storage_context=storage_context_graph)

INFO:llama_index.indices.loading:Loading all indices.
Loading all indices.


# Querying

In [7]:
query_engine_vector = vec_index.as_query_engine()
query_engine_graph = graph_index.as_query_engine()

In [8]:
query = "what is the difference between langchain and langsmith ?"

In [9]:
response_vector = str(query_engine_vector.query(query))
response_graph = query_engine_graph.query(query)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


INFO:httpx:HTTP Request: POST https://doc-ocr-adl.openai.azure.com//openai/deployments/chat_model/chat/completions?api-version=2023-05-15 "HTTP/1.1 200 OK"
HTTP Request: POST https://doc-ocr-adl.openai.azure.com//openai/deployments/chat_model/chat/completions?api-version=2023-05-15 "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://doc-ocr-adl.openai.azure.com//openai/deployments/chat_model/chat/completions?api-version=2023-05-15 "HTTP/1.1 200 OK"
HTTP Request: POST https://doc-ocr-adl.openai.azure.com//openai/deployments/chat_model/chat/completions?api-version=2023-05-15 "HTTP/1.1 200 OK"


In [ ]:
print("vector db response : {}".format())